# CEE 598: Uncertainty Quantification
## Assignment Six
### Noah Garfinkle (garfink2@illinois edu)
### 03 May 2020

## Imports and Set Up

In [5]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import pytest
import chaospy as cp
import pandas as pd

## References

In [6]:
# https://chaospy.readthedocs.io/en/master/tutorial.html
# https://chaospy.readthedocs.io/en/master/_modules/chaospy/quadrature/clenshaw_curtis.html
# https://chaospy.readthedocs.io/en/master/quadrature.html#module-chaospy.quadrature.clenshaw_curtis
# https://chaospy.readthedocs.io/en/master/quadrature.html#module-chaospy.quadrature.sparse_grid
# https://chaospy.readthedocs.io/en/master/recipes/galerkin.html

## General Approach

Building off of assignment five, I have opted to again utilize the third path of confirming an existing library.  As with Assignment Five, I have opted to utilize Chaospy for Python.

In order to evaluate the integral, I have opted to utilize an approach similar to Monte Carlo approximations of the integral, represented by

$$\int{f(X)} \approx \sum_{n=1}^{N}W_nf(X_n)$$

Where $N$ is the number of samples, $X_n$ is each individual sample, $f(X_n)$ is the evaluation of the target function at the sample coordinates, and $W_n$ is the weight of the sample as defined by the rule utilized.

## Problem 1.
Consider the following target function

$$f(x)=f(x_1,x_2,...,x_5)=\prod_{i=1}^5sin(ix_i)$$

where $x_i \in [0,1],\forall_i$.  We would like to approximate $\int_{[0,1]^5}f(x)dx$ using quadrature.  Compare the following multidimensional quadrature rules:

I have assumed that $x_i$ are each uniformally distributed.

In [47]:
def func(x):
    x1,x2,x3,x4,x5 = x
    value = np.sin(x1) * np.sin(2*x2) * np.sin(3*x3) * np.sin(4*x4) * np.sin(5*x5)
    return value

In [48]:
distribution_x = cp.J(cp.Uniform(0,1),cp.Uniform(0,1),cp.Uniform(0,1),cp.Uniform(0,1),cp.Uniform(0,1))

### Problem 1a. Tensor product using Clenshaw-Curtis (CC) points at levels 1,2, and 3.

Note: Clenshaw-Curtis points and their weights are given on Slide 15 of Lecture 20

In [110]:
order = 3
sampleRule = "clenshaw_curtis"
sparse = False
growth = False
abscissas, weights = cp.generate_quadrature(order, distribution_x, rule=sampleRule,sparse=sparse,growth=growth)
samples = abscissas.T

In [111]:
integral = 0.0
for i,sample in enumerate(samples):
    weight = weights[i]
    integral += weight * func(sample)

In [115]:
def calculateIntegral(order,sampleRule="clenshaw_curtis",sparse=False,growth=False):
    abscissas, weights = cp.generate_quadrature(order, distribution_x, rule=sampleRule,sparse=sparse,growth=growth)
    samples = abscissas.T
    integral = 0.0
    for i,sample in enumerate(samples):
        weight = weights[i]
        integral += weight * func(sample)
    return integral

In [116]:
calculateIntegral(3)

0.009638876045397832

In [120]:
orders = []
integrals = []
rules = []
sparses = []
for order in [1,2,3]:
    orders.append(order)
    integrals.append(calculateIntegral(order))
    rules.append("clenshaw_curtis")
    sparses.append(False)
df = pd.DataFrame({"Order":orders,"Integral":integrals,"Rule":rules,"Sparse":sparses})

In [121]:
df

,Order,Integral,Rule,Sparse
0,1,0.002449,clenshaw_curtis,False
1,2,0.025902,clenshaw_curtis,False
2,3,0.009639,clenshaw_curtis,False


In [114]:
samples

array([[0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.25],
       [0.  , 0.  , 0.  , 0.  , 0.75],
       ...,
       [1.  , 1.  , 1.  , 1.  , 0.25],
       [1.  , 1.  , 1.  , 1.  , 0.75],
       [1.  , 1.  , 1.  , 1.  , 1.  ]])

### Problem 1b. Sparse grid using CC points at levels 1, 2, and 3.

### Problem 1c. Sparse grid using Gauss-Legendre quadrature points with the number of points in each dimension being equal to 1, 2, and 3. Compare it with one that does so using sparse grid quadratures at levels 1, 3, and 5.

## Problem 2.
Using the Stochastic Galerkin approach, find a PCE approximation for $y$ given by

$$ay=bc+d$$

where $a=3-0.4\Psi_2(z_1), b=-5+0.4\Psi_1(z_2), c=3+0.02\Psi_1(z_3), and d=11-3\Psi_1(z_3)$ are four random parameters, where ${z_i}$ are independent uniform random variables in $[0,1]$ and ${\Psi_i}$ are Legendre polynomials.